<a href="https://colab.research.google.com/github/panchambanerjee/code-carbon-expts/blob/main/code_carbon_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers datasets accelerate huggingface_hub codecarbon torch comet_ml -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.2/492.2 kB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.8/175.8 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.6/549.6 kB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 84.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [13]:
%%writefile comet_codecarbon_mnist.py

from comet_ml import Experiment  # isort:skip

import tensorflow as tf

# Assumes Comet variable environment configuration.
# See here to get your API_KEY:
# https://www.comet.ml/user/settings/account#section-DEVELOPER_INFORMATION
# And here for setup information:
# https://www.comet.ml/docs/python-sdk/advanced/#experiment-configuration-parameters
experiment = Experiment(api_key='')

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10),
    ]
)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer="adam", loss=loss_fn, metrics=["accuracy"])

model.fit(x_train, y_train, epochs=10)

Overwriting comet_codecarbon_mnist.py


In [ ]:
!python comet_codecarbon_mnist.py

2023-07-26 18:54:28.138279: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-26 18:54:29.108808: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[codecarbon INFO @ 18:54:31] [setup] RAM Tracking...
[codecarbon INFO @ 18:54:31] [setup] GPU Tracking...
[codecarbon INFO @ 18:54:31] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 18:54:31] [setup] CPU Tracking...
[codecarbon WARNING @ 18:54:31] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 18:54:33] We saw that you have a Intel(R) Xeon(R) CPU @ 2.30GHz but we don't know it. Please contact us.
[codecarbon INFO @ 18:54:33] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.30GHz
[codecarbon INFO @ 18:5

In [2]:
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
import gc
import torch

# gc.collect()
# torch.cuda.empty_cache()

In [3]:
gc.collect()

60

In [4]:
from codecarbon import EmissionsTracker

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

## Bert Base Uncased

In [1]:
tracker = EmissionsTracker()
tracker.start()

# Compute intensive code goes here

ds = load_dataset("imdb")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


small_train_dataset = ds["train"].shuffle(seed=42).\
select(range(1000)).map(tokenize_function, batched=True)

small_eval_dataset = ds["test"].shuffle(seed=42).\
select(range(1000)).map(tokenize_function, batched=True)

training_args = TrainingArguments(
    output_dir="/content/code-carbon-classification",
    num_train_epochs=1,
    push_to_hub=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset
)

trainer.train()

# Compute intensive code stops here

tracker.stop()

NameError: ignored

In [11]:
%%writefile bert-base-cased-codecarbon-comet-run.py

from comet_ml import Experiment

experiment = Experiment(api_key='')

from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Compute intensive code goes here

ds = load_dataset("imdb")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


small_train_dataset = ds["train"].shuffle(seed=42).\
select(range(1000)).map(tokenize_function, batched=True)

small_eval_dataset = ds["test"].shuffle(seed=42).\
select(range(1000)).map(tokenize_function, batched=True)

training_args = TrainingArguments(
    output_dir="output_dir",
    num_train_epochs=10,
    push_to_hub=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset
)

trainer.train()

Overwriting bert-base-cased-codecarbon-comet-run.py


In [12]:
!python bert-base-cased-codecarbon-comet-run.py

[codecarbon INFO @ 18:39:11] [setup] RAM Tracking...
[codecarbon INFO @ 18:39:11] [setup] GPU Tracking...
[codecarbon INFO @ 18:39:11] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 18:39:11] [setup] CPU Tracking...
[codecarbon WARNING @ 18:39:11] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 18:39:13] We saw that you have a Intel(R) Xeon(R) CPU @ 2.30GHz but we don't know it. Please contact us.
[codecarbon INFO @ 18:39:13] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.30GHz
[codecarbon INFO @ 18:39:13] >>> Tracker's metadata:
[codecarbon INFO @ 18:39:13]   Platform system: Linux-5.15.109+-x86_64-with-glibc2.35
[codecarbon INFO @ 18:39:13]   Python version: 3.10.6
[codecarbon INFO @ 18:39:13]   CodeCarbon version: 2.2.7
[codecarbon INFO @ 18:39:13]   Available RAM : 12.678 GB
[codecarbon INFO @ 18:39:13]   CPU count: 2
[codecarbon INFO @ 18:39:13]   CPU model: Intel(R) Xeon(R) CPU @ 2.30GHz
[codecarbon INFO @ 18:39:13]   GPU 